In [2]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split

In [3]:
#Loading the data with pandas
math = pd.read_csv("../Data/math.csv")
read = pd.read_csv("../Data/read.csv")
business = pd.read_csv("../Data/BUSSINES.csv")
geo = pd.read_csv("../Data/geo.csv", low_memory=False)
funding = pd.read_csv("../Data/FUNDING.csv")
income = pd.read_csv("../Data/income.csv")
teachers = pd.read_csv("../Data/Teacher Ratios.csv")
title1 = pd.read_csv("../Data/title1.csv")
#Set indexs
math = math.set_index("math_ncessch")
read = read.set_index("read_ncessch")
business = business.set_index("zip_code")
funding["ncesid"] = pd.to_numeric(funding["ncesid"], errors="coerce")
funding = funding.set_index("ncesid")
geo = geo.set_index("ncessch")
income = income.set_index("zipcode")
teachers["ncessch"] = teachers["ncessch"].astype(np.int64)
teachers = teachers.set_index("ncessch")
title1["ncessch"] = title1["ncessch"].astype(np.int64)
title1 = title1.set_index("ncessch")

In [4]:
combined = math[['math_leaid','math_all_grades_numvalid','math_all_grades_pctprof_low','math_all_grades_pctprof_high']].join([read[['read_all_grades_numvalid','read_all_grades_pctprof_low','read_all_grades_pctprof_high']],geo[['zip','locale','lat','lon']]], how="inner")
combined = combined.join(business[['num_establishments',"num_paid_employees", "first_quarter_payroll", "annual_payroll"]], how="inner", on="zip")
combined = combined.join(funding.drop(["idcensus","name","conum","csa","cbsa","enroll"],1), how="inner", on="math_leaid")
combined = combined.join(income.drop(["statefips",'state'], 1), how="inner", on="zip")
combined = combined.join(teachers["num_full_time"], how="inner")
#combined = combined.join(title1[["title_i_status","title_i_eligibility","school_wide_title_i_eligibility","ntnl_school_lunch_program_status"]], how="inner")
combined = combined.drop(['zip','num_paid_employees',"first_quarter_payroll", "annual_payroll"],1)
combined = combined[combined['math_all_grades_pctprof_low'] >=0]
combined = combined[combined['read_all_grades_pctprof_low'] >=0]
math_y = (combined['math_all_grades_pctprof_low']+combined['math_all_grades_pctprof_high'])/2
read_y = (combined['read_all_grades_pctprof_low']+combined['read_all_grades_pctprof_high'])/2
math_x = combined.drop(["math_all_grades_pctprof_low","math_all_grades_pctprof_high","read_all_grades_pctprof_low","read_all_grades_pctprof_high"],1)
read_x = combined.drop(["math_all_grades_pctprof_low","math_all_grades_pctprof_high","read_all_grades_pctprof_low","read_all_grades_pctprof_high"],1)

In [6]:
list(set(math_y))

[0.5,
 2.0,
 2.5,
 4.0,
 5.0,
 6.0,
 3.0,
 7.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.5,
 44.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 84.5,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.5,
 97.0,
 99.5,
 98.0,
 24.5,
 34.5,
 8.0,
 45.0,
 49.5,
 54.5,
 64.5,
 69.5,
 74.5,
 75.5,
 88.0]

In [7]:
math_X_train_val, math_X_test, math_y_train_val, math_y_test = train_test_split(math_x, math_y, test_size = .2, train_size = .8, random_state =42 )
math_X_train, math_X_val, math_y_train, math_y_val = train_test_split(math_X_train_val, math_y_train_val, test_size = .3, train_size = .7, random_state =42 )

In [8]:
#depth = 0
#max_score = 0
#for i in range(1,20):
#    print("running with max depth ", i)
#    rfr = RandomForestRegressor(n_estimators=100, max_depth=i, random_state=42)
#    trained = rfr.fit(math_X_train, math_y_train)
#    score = rfr.score(math_X_val, math_y_val)
#    print("validation score", score)
#    if score > max_score:
#        depth = i
#        max_score = score
rfr = RandomForestRegressor(n_estimators=100, max_depth=19, random_state=42)
trained = rfr.fit(math_X_train, math_y_train)
print(rfr.score(math_X_test, math_y_test))

0.6696004978248256


In [11]:
def calculateDelta(x_test, y_test,estimator):
    predictions = np.array(estimator.predict(x_test))
    return np.mean(np.absolute(predictions - y_test))
calculateDelta(math_X_test, math_y_test, rfr)

10.09386333753234

In [14]:
rfr.feature_importances_

array([0.15058525, 0.07122716, 0.03615167, 0.00409731, 0.06020189,
       0.0396446 , 0.00562029, 0.0005976 , 0.00135583, 0.00260973,
       0.00428746, 0.00263914, 0.00283146, 0.00127622, 0.002011  ,
       0.00573749, 0.08313095, 0.0043862 , 0.00173413, 0.00203861,
       0.00147058, 0.00047799, 0.00322704, 0.00550529, 0.00376374,
       0.00329645, 0.0006818 , 0.00050475, 0.00066513, 0.00087688,
       0.00073341, 0.00066024, 0.00097285, 0.0006331 , 0.00150931,
       0.00207221, 0.00321168, 0.00113227, 0.00100464, 0.0014554 ,
       0.00334837, 0.00674272, 0.00234948, 0.00282615, 0.00139347,
       0.00271807, 0.        , 0.05867767, 0.07632806, 0.00357795,
       0.00420971, 0.00622212, 0.00606565, 0.00342368, 0.03654697,
       0.00278188, 0.00340185, 0.00660621, 0.00269298, 0.00507965,
       0.00707661, 0.00572073, 0.00672433, 0.00472433, 0.0050331 ,
       0.00582825, 0.00084764, 0.00285752, 0.00401677, 0.00975723,
       0.00127024, 0.00252083, 0.00142323, 0.00084762, 0.00142